In [7]:
import os
from PIL import Image
import pandas as pd
import torch

dataset_dir = '/Users/anakhaganesh/Documents/MIT/6.8301 Spring 2024/Final Project/data/BCI_dataset/'
x_train_dir = 'HE/train/'
x_test_dir = 'HE/test/'
y_train_dir = 'IHC/train/'
y_test_dir = 'IHC/test/'

x_train = []
y_train = []
for filename in os.listdir(dataset_dir + x_train_dir):
    if filename.endswith('.png'):
        image_x_train = Image.open(os.path.join(dataset_dir + x_train_dir, filename))
        x_train.append(image_x_train)

for filename in os.listdir(dataset_dir + x_train_dir):
    if filename.endswith('.png'):        
        image_y_train = Image.open(os.path.join(dataset_dir + y_train_dir, filename))
        y_train.append(image_y_train)

print(x_train)
print(y_train)

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x150383170>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15039DFD0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186A960>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186A9F0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186AA80>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186AB10>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186ABA0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186AC30>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186ACC0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186AD50>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186ADE0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x15186AE70>, <PIL.JpegImageP

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # Contracting Path (Encoder)
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Add more convolutional layers as needed...
        )

        # Expansive Path (Decoder)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, out_channels, kernel_size=2, stride=2),
            nn.Sigmoid()  # Adjust activation function based on your task
        )

    def forward(self, x):
        # Contracting Path
        encoder_output = self.encoder(x)
        
        # Expansive Path
        output = self.decoder(encoder_output)
        
        return output

# Example usage:
# Define input and output channels
in_channels = 3  # Assuming RGB images
out_channels = 1  # Assuming grayscale segmentation mask

# Create an instance of UNet model
model = UNet(in_channels, out_channels)

# Print model summary
print(model)


UNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(64, 1, kernel_size=(2, 2), st

In [27]:
# Load training images and labels
x_train_paths = [os.path.join(dataset_dir, x_train_dir, filename) for filename in os.listdir(dataset_dir + x_train_dir) if filename.endswith('.png')]
y_train_paths = [os.path.join(dataset_dir, y_train_dir, filename) for filename in os.listdir(dataset_dir + y_train_dir) if filename.endswith('.png')]
x_train = load_images(x_train_paths)
y_train = load_images(y_train_paths)

# Define input and output channels
in_channels = 3  # RGB images
out_channels = 1  # Grayscale segmentation mask

# Create DataLoader for training data
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

NameError: name 'load_images' is not defined

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

In [ ]:
# Define loss function and optimizer
model = UNet(in_channels=3, out_channels=1)  # Assuming 3 input channels (RGB) and 1 output channel (segmentation mask)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Optionally, evaluate the model on a validation set and print metrics
# Define validation DataLoader, forward pass through the model, and compute metrics